# AI-Mind

[Minds](https://mindsdb.com/minds) are AI systems provided by [MindsDB](https://mindsdb.com/) that work similarly to large language models (LLMs) but go beyond by answering any question from any data. 

This is accomplished by selecting the most relevant data for an answer using parametric search, understanding the meaning and providing responses within the correct context through semantic search, and finally, delivering precise answers by analyzing data and using machine learning (ML) models.

The `AIMindTool` provides access to these Minds through LangChain, allowing their capabilities to be harnessed seamlessly. 

## Overview

### Integration details

- TODO: Make sure links and features are correct

| Class | Package | Serializable | [JS support](https://js.langchain.com/docs/integrations/tools/langchain_minds) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [AIMindTool](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.langchain_minds.tool.Minds.html) | [langchain-community](https://api.python.langchain.com/en/latest/community_api_reference.html) | beta/❌ | ✅/❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-community?style=flat-square&label=%20) |

### Tool features

- TODO: Add feature table if it makes sense


## Setup

To access Minds via the `AIMindTool`, you'll need to create a Minds account, get an API key, and install the `langchain-minds` integration package.

In [ ]:
%pip install --quiet -U langchain-minds

### Credentials

In [2]:
import getpass
import os

if not os.environ.get("MINDS_API_KEY"):
    os.environ["MINDS_API_KEY"] = getpass.getpass("MINDS API key:\n")

## Instantiation

In [1]:
from langchain_minds import AIMindDataSource, AIMindAPIWrapper, AIMindTool


# Create a data source that your Mind will have access to.
# To configure additional data sources, simply create additional instances of AIMindDataSource and pass it to the wrapper below.
datasource = AIMindDataSource(
    description='house sales data',
    engine='postgres',
    connection_data={
        'user': 'demo_user',
        'password': 'demo_password',
        'host': 'samples.mindsdb.com',
        'port': 5432,
        'database': 'demo',
        'schema': 'demo_data'
    },
    tables=['house_sales']
)

# Create the wrapper for the Minds API by passing in the data sources created above.
api_wrapper = AIMindAPIWrapper(
    datasources=[datasource]
)

# Create the tool by simply passing in the API wrapper from before.
tool = AIMindTool(
    api_wrapper=api_wrapper
)

ModuleNotFoundError: No module named 'langchain_minds'

## Invocation

### Invoke directly with args

The only required argument to query the `AIMindTool` is `query`, which is a question for the Mind in plain English.

In [ ]:
tool.invoke({"query": "How many three-bedroom houses were sold in 2008?"})

### Invoke with ToolCall

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned.

In [ ]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "How many three-bedroom houses were sold in 2008?"},
    "id": "1",
    "name": tool.name,
    "type": "tool_call",
}
tool.invoke(model_generated_tool_call)

## Chaining

- TODO: Add user question and run cells

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />


In [16]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("...")

## API reference

For detailed documentation of all Minds features and configurations head to the API reference: https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.langchain_minds.tool.Minds.html